In [1]:
from sklearn import svm
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.metrics import classification_report

In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [18]:
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
# Extract features using VGG16
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(r"C:\Users\HP\OneDrive\Desktop\data\data\train", target_size=(224, 224), batch_size=32, class_mode=None, shuffle=False)
train_features = base_model.predict(train_generator)

Found 1951 images belonging to 4 classes.


D:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 167s 3s/step


In [27]:
# Define SVM classifier
svm_classifier = svm.SVC(kernel='linear', C=1.0)

In [28]:
# Flatten features
train_features_flatten = train_features.reshape(train_features.shape[0], -1)

In [30]:
# Train SVM classifier
svm_classifier.fit(train_features_flatten, train_generator.classes)

SVC(kernel='linear')

In [32]:
# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(r"C:\Users\HP\OneDrive\Desktop\data\data\val", target_size=(224, 224), batch_size=32, class_mode=None, shuffle=False)
test_features = base_model.predict(test_generator)
test_features_flatten = test_features.reshape(test_features.shape[0], -1)
predictions = svm_classifier.predict(test_features_flatten)

Found 324 images belonging to 4 classes.


D:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step


In [37]:
# Print classification report
print(classification_report(test_generator.classes, predictions, target_names=test_generator.class_indices))

                       precision    recall  f1-score   support

 diseased cotton leaf       0.98      0.93      0.95        55
diseased cotton plant       0.99      0.95      0.97       101
    fresh cotton leaf       0.93      1.00      0.96        80
   fresh cotton plant       0.96      0.97      0.96        88

             accuracy                           0.96       324
            macro avg       0.96      0.96      0.96       324
         weighted avg       0.96      0.96      0.96       324

